In [15]:
import pandas as pd
import numpy as np
claims_data = pd.read_excel("claims_data.xlsx")

In [21]:
claims_data.loc[claims_data["Notification_period"] < 0, "Notification_period"] = np.nan
claims_data.loc[claims_data["PH_considered_TP_at_fault"] == "#", "PH_considered_TP_at_fault"] = np.nan

In [24]:
claims_data.drop(columns=["Claim Number", "date_of_loss", "Loss_code", "Loss_description", "Capped Incurred"], inplace=True)

In [25]:
claims_data

,Notifier,Notification_period,Inception_to_loss,Location_of_incident,Weather_conditions,Vehicle_mobile,Time_hour,Main_driver,PH_considered_TP_at_fault,Vechile_registration_present,...,TP_region_north,TP_region_northw,TP_region_outerldn,TP_region_scotland,TP_region_southe,TP_region_southw,TP_region_wales,TP_region_westmid,TP_region_yorkshire,Incurred
0,PH,22.0,13,Main Road,NORMAL,Y,10,Other,n/k,1,...,0,1,0,0,0,0,0,0,0,0.000000
1,CNF,1.0,9,Main Road,WET,Y,18,Other,n/k,1,...,0,0,0,0,1,0,0,0,0,2801.308013
2,CNF,5.0,17,Main Road,WET,Y,16,Y,n/k,1,...,0,1,0,0,0,0,0,0,0,1220.870390
3,CNF,1.0,23,Main Road,N/K,Y,14,Other,n/k,1,...,0,0,0,0,0,0,0,1,0,3529.868026
4,CNF,1.0,48,Other,N/K,N,9,Other,n/k,1,...,0,0,0,0,0,0,0,0,0,3155.987923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7686,Other,1.0,83,Main Road,NORMAL,N,16,Other,N,1,...,0,1,0,0,0,0,0,0,0,702.814301
7687,Other,0.0,25,Minor Road,NaN,Y,14,Y,N,1,...,0,0,0,0,0,0,0,0,2,42980.618893
7688,Other,0.0,60,Minor Road,NORMAL,Y,9,Other,N,1,...,0,0,0,0,1,0,0,0,0,5174.800838
7689,Other,1.0,253,Minor Road,NORMAL,N,19,Other,N,1,...,0,0,0,0,0,0,0,0,0,30072.417085


In [26]:
from sklearn.model_selection import train_test_split

In [31]:
train, temp = train_test_split(claims_data, test_size=0.25, random_state=32, shuffle=True)
val, test = train_test_split(temp, test_size=0.4, random_state=32, shuffle=True)

# Display the results
print("Training data size:", len(train) / (len(train) + len(val) + len(test)) )
print("Validation data size:", len(val) / (len(train) + len(val) + len(test)))
print("Testing data size:", len(test)/ (len(train) + len(val) + len(test)))

Training data size: 0.7499674944740606
Validation data size: 0.14991548563255752
Testing data size: 0.10011701989338187


In [35]:
X_train = train.drop(columns="Incurred")
y_train = train["Incurred"]

In [36]:
from collections import namedtuple
DataSet = namedtuple('DataSet', ['features', 'target'])

train_set = DataSet(features=train.drop(columns="Incurred"), target=train["Incurred"])
val_set = DataSet(features=val.drop(columns="Incurred"), target=y_val)
test_set = DataSet(features=X_test, target=y_test)



In [ ]:
#print hello
